In [2]:
!python -m pip install --upgrade --quiet  pinecone-client pinecone-text pinecone-notebooks

In [3]:
api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [5]:
from langchain.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

pc=Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )

In [6]:
index=pc.Index(index_name)
index


In [7]:
## vector embedding and sparse matrix
import os
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedding

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [10]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

bm25_encoder.fit(sentences)
## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:04<00:00,  1.60s/it]


In [11]:
retriever=PineconeHybridSearchRetriever(embeddings=embedding,sparse_encoder=bm25_encoder,index=index)

In [12]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000002A5CE636F80>, index=<pinecone.data.index.Index object at 0x000002A5B7B59750>)

In [13]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:17<00:00, 17.33s/it]


In [14]:
retriever.invoke("what city do i visit?")

[Document(metadata={'score': 0.245233506}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.21958217}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.197595611}, page_content='In 2023, I visited Paris')]